# 12.U_CV_JPortilla_12(Object_tracking)

## OBJECT TRACKING

In [1]:
'''
Object Tracking techniques:
    -> Optical Flow
    -> Meanshift and CamShift
    -> OpenCV built-in Tracking APIs
'''

'\nObject Tracking techniques:\n    -> Optical Flow\n    -> Meanshift and CamShift\n    -> OpenCV built-in Tracking APIs\n'

### Optical Flow

In [2]:
'''
-> Optical flow is the pattern of apparent motion if image objects between two consecutive frames caused by the movement of object or camera
-> (Our) Optical Flow analysis here have few assumptions:
    --> The pixel intentsities of an object do not change between consecutive feames(Ex you're not trying to track a blinking bulb)
    --> Neighbouring pixels have similat motion
-> The optical glow methods in OpenCV will first take in a given set of points (user supplied) and a frame
-> Then it will attempt to find those points in the next frame
-> It is up to the user to supply the points to track
-> It uses LUCAS-KANADE(p: kana-de) function. The Lucas-Kandde function computes optical flow of a sparse 
   feature set(tracks the specified points only; not all points in the video)
-> Gunner Farneback's algorithm(also built into OpenCV) uses Dense Optical flow(calclulates flow of all points in an image); it will color points black if no flow 
   is detected
'''

"\n-> Optical flow is the pattern of apparent motion if image objects between two consecutive frames caused by the movement of object or camera\n-> (Our) Optical Flow analysis here have few assumptions:\n    --> The pixel intentsities of an object do not change between consecutive feames(Ex you're not trying to track a blinking bulb)\n    --> Neighbouring pixels have similat motion\n-> The optical glow methods in OpenCV will first take in a given set of points (user supplied) and a frame\n-> Then it will attempt to find those points in the next frame\n-> It is up to the user to supply the points to track\n-> It uses LUCAS-KANADE(p: kana-de) function. The Lucas-Kandde function computes optical flow of a sparse \n   feature set(tracks the specified points only; not all points in the video)\n-> Gunner Farneback's algorithm(also built into OpenCV) uses Dense Optical flow(calclulates flow of all points in an image); it will color points black if no flow \n   is detected\n"

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image 

''' 
-> Window size parameter:
   --> smaller window:more sensitive to noise; miss larger movement
   --> larger window: miss smaller movements
-> max Level : Lucas Kanade algorithm can be used with Image Pyramid
'''

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking1.png', height=100, width=300)

#### Capture/read video file

In [ ]:
capture = cv2.VideoCapture('../../data/raw/Computer-Vision-with-Python_Joseph_Portilla/DATA/moving_car2.mov')

return_value, previous_frame = capture.read()

plt.imshow(previous_frame)

In [ ]:
#### Create Markers(using goodFeaturesToTrack)

In [ ]:
'''
Docstring:
goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance[, corners[, mask[, blockSize[, useHarrisDetector[, k]]]]]) -> corners
.   @brief Determines strong corners on an image.
.   The function finds the most prominent corners in the image or in the specified image region 
.   -   Function calculates the corner quality measure at every source image pixel using the #cornerMinEigenVal or #cornerHarris .
.   -   Function performs a non-maximum suppression (the local maximums in *3 x 3* neighborhood are retained).
.   -   The corners with the minimal eigenvalue less than \f$\texttt{qualityLevel} \cdot \max_{x,y} qualityMeasureMap(x,y)\f$ are rejected.
.   -   The remaining corners are sorted by the quality measure in the descending order.
.   -   Function throws away each corner for which there is a stronger corner at a distance less than  maxDistance.
.   
.   The function can be used to initialize a point-based tracker of an object.
.  
.   @param image Input 8-bit or floating-point 32-bit, single-channel image.
.   @param corners Output vector of detected corners.
.   @param maxCorners Maximum number of corners to return. If there are more corners than are found,
.   the strongest of them is returned. `maxCorners <= 0` implies that no limit on the maximum is set
.   and all detected corners are returned.
.   @param qualityLevel Parameter characterizing the minimal accepted quality of image corners. The
.   parameter value is multiplied by the best corner quality measure, which is the minimal eigenvalue
.   (see #cornerMinEigenVal ) or the Harris function response (see #cornerHarris ). The corners with the
.   quality measure less than the product are rejected. For example, if the best corner has the
.   quality measure = 1500, and the qualityLevel=0.01 , then all the corners with the quality measure
.   less than 15 are rejected.
.   @param minDistance Minimum possible Euclidean distance between the returned corners.
.   @param mask Optional region of interest. If the image is not empty (it needs to have the type
.   CV_8UC1 and the same size as image ), it specifies the region in which the corners are detected.
.   @param blockSize Size of an average block for computing a derivative covariation matrix over each
.   pixel neighborhood. See cornerEigenValsAndVecs .
.   @param useHarrisDetector Parameter indicating whether to use a Harris detector (see #cornerHarris)
.   or #cornerMinEigenVal.
.   @param k Free parameter of the Harris detector.
'''

In [ ]:
# Using goodFeaturesToTrack to get the markers to track
previous_frame_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

corner_track_params = dict(maxCorners=8, qualityLevel=0.1,minDistance=20, blockSize=10)

previous_points = cv2.goodFeaturesToTrack(previous_frame_gray, mask=None, **corner_track_params)
previous_points_integer = np.int0(previous_points)

# Display the markers
previous_frame_copy = previous_frame.copy()
for i in previous_points_integer:
    x, y = i.ravel() # flatten the array
    #print(x,y)
    cv2.circle(previous_frame_copy, (x,y), radius=10, color=[255,0,0], thickness=-1)
plt.imshow(previous_frame_copy)

### Lucas Kanade algorithm (sparse)

### Parameters for Lucas Kanade Optical Flow

Detect the motion of specific points or the aggregated motion of regions by modifying the winSize argument. This determines the integration window size. Small windows are more sensitive to noise and may miss larger motions. Large windows will “survive” an occlusion.

The integration appears smoother with the larger window size.

criteria has two here - the max number (10 above) of iterations and epsilon (0.03 above). More iterations means a more exhaustive search, and a smaller epsilon finishes earlier. These are primarily useful in exchanging speed vs accuracy, but mainly stay the same.

When maxLevel is 0, it is the same algorithm without using pyramids (ie, calcOpticalFlowLK). Pyramids allow finding optical flow at various resolutions of the image. 

In [ ]:
# cv2.TERM_CRITERIA_COUNT is the number of iterations (larger value -> more iterations -> more time); here we use 10
# cv2.TERM_CRITERIA_EPS is the epsolon(?) values; here we use 0.03 (smaller value -> less time)
# These 2 parameters decide speed v/s accuracy

#corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7)
lucas_kanade_params = dict(winSize=(200,200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking2.png', height=100, width=300)

In [ ]:
mask = np.zeros_like(previous_frame)
previous_frame_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

while True:
    return_value, frame = capture.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame_gray = frame.copy()
    
    next_points, status, error = cv2.calcOpticalFlowPyrLK(previous_frame_gray, frame_gray, prevPts=previous_points, nextPts=None, **lucas_kanade_params)
    
    good_next_points = next_points[status == 1]
    good_prev_points = previous_points[status == 1]
    
    for i, (new, prev) in enumerate(zip(good_next_points, good_prev_points)):
        x_new,  y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        # tracking lines for the markers
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0,255,0), 3)
        frame = cv2.circle(frame, (x_new, y_new), 8, (0,0,255), -1)
        
    image = cv2.add(frame, mask)
    cv2.imshow('tracking', image) 
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    # replace previous_frame_gray with new frame_gray, previous_points with good_next_points for next iteration
    previous_frame_gray = frame_gray.copy()
    previous_points = good_next_points.reshape(-1,1,2)
    
    
capture.release()
cv2.destroyAllWindows()
# OUTPUT screen shot is shown below(computer_vision7_object_tracking2.png)

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking2.png', height=100, width=300)

In [ ]:
plt.imshow(frame_gray)

In [ ]:
frame_gray.shape

In [ ]:
frame_gray

In [ ]:
np.min(frame_gray), np.max(frame_gray)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image 
from datetime import datetime
from matplotlib import cm

capture = cv2.VideoCapture('../../data/raw/Computer-Vision-with-Python_Joseph_Portilla/DATA/moving_car3.mov')
return_value, previous_frame = capture.read()
#plt.imshow(previous_frame)

# Using goodFeaturesToTrack to get the markers to track
previous_frame_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

corner_track_params = dict(maxCorners=8, qualityLevel=0.1,minDistance=20, blockSize=10)

previous_points = cv2.goodFeaturesToTrack(previous_frame_gray, mask=None, **corner_track_params)
previous_points_integer = np.int0(previous_points)

# Display the markers
previous_frame_copy = previous_frame.copy()
for i in previous_points_integer:
    x, y = i.ravel() # flatten the array
    #print(x,y)
    cv2.circle(previous_frame_copy, (x,y), radius=10, color=[255,0,0], thickness=-1)
plt.imshow(previous_frame_copy)

In [ ]:
lucas_kanade_params = dict(winSize=(200,200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
mask = np.zeros_like(previous_frame)
previous_frame_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

while True:
    return_value, frame = capture.read()
    if not return_value:
        break
        
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame_gray = frame.copy()
    
    next_points, status, error = cv2.calcOpticalFlowPyrLK(previous_frame_gray, frame_gray, prevPts=previous_points, nextPts=None, **lucas_kanade_params)
    
    good_next_points = next_points[status == 1]
    good_prev_points = previous_points[status == 1]
    
    for i, (new, prev) in enumerate(zip(good_next_points, good_prev_points)):
        x_new,  y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        # tracking lines for the markers
        #mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0,255,0), 1)
        frame = cv2.circle(frame, (x_new, y_new), 4, tuple(np.array(cm.tab10(0)[:3]) * 255), -1)
        
    image = cv2.add(frame, mask)
    cv2.imshow('tracking', image) 
    filename= datetime.now().strftime("%Y%m%d-%H%M%S")
    plt.imsave(filename, image)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        print('got here')
        break

    # replace previous_frame_gray with new frame_gray, previous_points with good_next_points for next iteration
    previous_frame_gray = frame_gray.copy()
    previous_points = good_next_points.reshape(-1,1,2)
    
    
capture.release()
cv2.destroyAllWindows()
# OUTPUT screen shot is shown below(computer_vision7_object_tracking3.png)

In [ ]:
def showImage(name, image):
    cv2.startWindowThread()
    cv2.imshow(name,image)
    # cv2.startWindowThread()
    cv2.waitKey()
    cv2.destroyAllWindows()
    for i in range (1,10):
        cv2.waitKey(1) 

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking3.png', height=100, width=300)

# Dense Optical Flow in OpenCV

calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags) -> flow

This function computes a dense optical flow using the Gunnar Farneback's algorithm.

Here are the parameters for the function and what they represent:
   

In [ ]:
import cv2 
import numpy as np

# Capture the frame
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

# Get gray scale image of first frame and make a mask in HSV color
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    # Check out the markdown text above for a break down of these paramters, most of these are just suggested defaults
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    
    # Color the channels based on the angle of travel
    # Pay close attention to your video, the path of the direction of flow will determine color!
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # Convert back to BGR to show with imshow from cv
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # Set the Previous image as the next iamge for the loop
    prvsImg = nextImg

    
cap.release()
cv2.destroyAllWindows()

In [ ]:
#OUTPUT shown below(computer_vision7_object_tracking6)
# Different colors when the object moves in different direction

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking6.png', height=100, width=300)

In [ ]:
#NOTES: Below images explain how vector flow info is converted into hue,saturation, value

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking4.png', height=100, width=300)

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking5.png', height=100, width=300)

## MEANSHIFT Alogorithm(similar to K-means)

In [ ]:
# Meanshift alogorithm is similar to k-means clustering, except that meanshift decides on the k value

In [ ]:
'''
1) Imagine we have a set of points and we wanted to assign them to clusters
2) We take all the data points and stack red and blue points on them(you cannot see red points underneath) (Initial state)
3) The direction to the closest cluster centroid is determined by where most of the points nearby are at(weighted mean is calculated)
4) So each iteration, each blue point will move closer to where the most points are at(which is or will lead to -> cluster centers)
5) The red and blue data points overlap completely in the initial state (before the meanshift algorithm starts)
6) At the end of iteration 1, all the blue points move towards the clusters (here it appears there will be 3 or 4 clusters)
7) At the end of iteration 2, both bottom clusters hae begun to reach convergence
8) At the end of iteration 3, meanshift found 3 clusters
9) After subsequent iterations, the cluster means have stopped moving
10) All the clusters have converged and there is no more movement
11) Meanshift has identified 3 clusters
'''

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking7.png', height=600, width=800)

In [ ]:
'''
How meanshift algorithm is applied to object tracking:
-> Meanshift can be given a target to track; it calculates the color histogram of the target area, and then keep sliding the tracking window
   to the closest match(the cluster center)
'''

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking8.png', height=400, width=600)

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking9.png', height=350, width=450)

## CAMSHIFT (Continuously adapted Meanshift)

In [ ]:
'''
Meanshift won't change the window size if the target moves away or toward the camera. We can use 
CAMshift to update the size of the window

1) First apply meanshift and perform the iterations of the meanshift
2) Once meanshift converges, we calculate a new region of interest. We do that by updating the size of the actual region of interst 
   and we also calculate the orientation of the best fitting elipse to the new region of interest.
3) Again, meanshift is applied with the new scaled search window in previous window location.
4) This process is continued until some sort of required accuracy is met
All this happends behind the scenes with OpenCV
'''

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking10.png', height=400, width=800)

In [ ]:
Image(filename='../images/UDEMY_computer_vision_Joseph_Portilla/computer_vision7_object_tracking11.png', height=150, width=500)

### TO DO: videos 62, 63, 64

In [ ]:
#END